In [2]:
import numpy as np
import chainer
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, FunctionNode, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import math

ImportError: No module named 'chainer'

In [67]:
class MulAdd(Function):
    def forward_cpu(self, inputs):
        x, y, z = inputs
        w = x * y + z
        return w,

    def backward_cpu(self, inputs, grad_outputs):
        x, y, z = inputs
        gw, = grad_outputs
        gx = y * gw
        gy = x * gw
        gz = gw
        print(y)
        return gx, gy, gz

In [148]:
x = Variable(np.random.uniform(-1, 1, (3, 2)).astype(np.float32))
y = Variable(np.random.uniform(-1, 1, (3, 2)).astype(np.float32))
z = Variable(np.random.uniform(-1, 1, (3, 2)).astype(np.float32))
w = MulAdd()(x, y, z)

def muladd(x, y, z):
    return MulAdd()(x, y, z)

print(x)
print(y)
w = muladd(x, y, z)
print(w)
e = F.batch_l2_norm_squared(w)
print(e)
e.backward()
print(x.grad)

variable([[ 0.47544184 -0.54639834]
          [-0.7283307  -0.3504386 ]
          [-0.33281076  0.95605004]])
variable([[ 0.36341515  0.9123046 ]
          [-0.26539513 -0.11405425]
          [-0.8109033  -0.87408   ]])
variable([[ 1.044728   -1.02573   ]
          [-0.14319816  0.3267007 ]
          [ 0.14952986 -0.18939847]])
variable([2.1435788  0.12723905 0.05823096])


AttributeError: 'NoneType' object has no attribute 'reshape'

In [68]:
x = Variable(np.array([5], dtype = 'float'))
y = Variable(np.array([3], dtype = 'float'))
z = Variable(np.array([2], dtype = 'float'))
w = x * y + z

w = muladd(x, y, z)
w.backward()

[3.]


In [6]:
z = F.exp(x)
z.backward()
print(z)

variable([148.4131591])


In [3]:
class Oplus(Function):
    def forward_cpu(self, inputs):
        t1, t2 = inputs
        #self.retain_inputs((0, 1))
        cos1 = math.cos(t1[2])
        sin1 = math.sin(t1[2])
        x = cos1 * t2[0] - sin1 * t2[1] + t1[0]
        y = sin1 * t2[0] + cos1 * t2[1] + t1[1]
        t = ( t1[2] + t2[2] + np.pi) % (2 * np.pi ) - np.pi
        ret = np.array([x,y,t], dtype=t1.dtype),
        #print('ret:' + str(ret))
        return ret
    
    def backward_cpu(self, inputs, grad_outputs):
        #t1, t2 = self.retained_inputs()
        t1, t2 = inputs
        gw, = grad_outputs
        #print('go')
        #print(grad_outputs)
        cos1 = math.cos(t1[2])
        sin1 = math.sin(t1[2])
        dx1 = np.array([[1., 0., -sin1 * t2[0] - cos1 * t2[1]],
                        [0., 1.,  cos1 * t2[0] - sin1 * t2[1]],
                        [0., 0., 1.]], dtype=gw.dtype)
        #print('dx1')
        #print(dx1)
        gw = gw.reshape(1,3)
        #print(gw.shape)
        #print(dx1.shape)
        d1 = np.squeeze(np.matmul(gw, dx1))
        dx2 = np.array([[cos1, -sin1, 0.],
                        [sin1, cos1, 0.],
                        [0, 0, 1.]], dtype=gw.dtype)
        d2 = np.squeeze(np.matmul(gw, dx2))
        return d1, d2

def oplus(x, y):
    return Oplus()(x,y)

In [5]:
x1 = Variable(np.array([1, 1, 1], dtype = 'float32'))
x2 = Variable(np.array([1, 2, 0], dtype = 'float32'))
z = oplus(x1,x2)
print(z)

z = z * np.array([1., 1., 0], dtype='float32')
e = F.matmul(z, F.transpose(z))
#e = F.average(z)
print(e)
e.backward()

func = e.creator
print(func.forward((x1.data, x2.data)))

print(x1.grad)

ep = 1e-5
x11 = x1 + np.array([ep, 0, 0], dtype='float')
z1 = oplus(x11, x2)
z1 = z1 * np.array([1., 1., 0], dtype='float')

e1 = F.matmul(z1, F.transpose(z1))
print('num dx1:' + str((e1-e)/ep))

x11 = x1 + np.array([0, ep, 0], dtype='float')
z1 = oplus(x11, x2)
z1 = z1 * np.array([1., 1., 0], dtype='float')
e1 = F.matmul(z1, F.transpose(z1))
print('num dy1:' + str((e1-e)/ep))

x11 = x1 + np.array([0, 0, ep], dtype='float')
z1 = oplus(x11, x2)
z1 = z1 * np.array([1., 1., 0], dtype='float')
e1 = F.matmul(z1, F.transpose(z1))
print('num dt1:' + str((e1-e)/ep))

print(x2.grad)

x22 = x2 + np.array([ep, 0, 0], dtype='float')
z1 = oplus(x1, x22)
z1 = z1 * np.array([1., 1., 0], dtype='float')

e1 = F.matmul(z1, F.transpose(z1))
print('num dx2:' + str((e1-e)/ep))

x22 = x2 + np.array([0, ep, 0], dtype='float')
z1 = oplus(x1, x22)
z1 = z1 * np.array([1., 1., 0], dtype='float')

e1 = F.matmul(z1, F.transpose(z1))
print('num dy2:' + str((e1-e)/ep))


x22 = x2 + np.array([0, 0, ep], dtype='float')
z1 = oplus(x1, x22)
z1 = z1 * np.array([1., 1., 0], dtype='float')

e1 = F.matmul(z1, F.transpose(z1))
print('num dt2:' + str((e1-e)/ep))

#z.grad = np.array([1.,1.,1.,], dtype='float32')
#f = lambda: func.forward((x1.data, x2.data))
#gx1, gx2 = gradient_check.numerical_grad(f, (x1.data, x2.data), (z.grad,), eps=1e-2)
#print(gx1)
#print(gx2)


x1 = Variable(np.array([1, 0, math.pi/2], dtype = 'float32'))
x2 = Variable(np.array([1, 0, math.pi/2], dtype = 'float32'))
x3 = Variable(np.array([1, 0, math.pi/2], dtype = 'float32'))
x4 = Variable(np.array([0.5, 0, math.pi/2], dtype = 'float32'))

for i in range(100):
    z1 = oplus(x1, x2)
    z2 = oplus(z1, x3)
    z = oplus(z2, x4)
    print(z)

    z = z * np.array([1., 1., 0], dtype='float32')
    e = F.matmul(z, F.transpose(z))
    #e = F.average(z)
    print(e)
    e.backward()
    #print(x1.grad)
    x1 = Variable(x1.data - 0.02 * x1.grad)
    x2 = Variable(x2.data - 0.02 * x2.grad)
    x3 = Variable(x3.data - 0.02 * x3.grad)
    #x4 = Variable(x4.data - 0.02 * x4.grad)
print(x1)
print(x2)
print(x3)
print(x4)

variable([-0.14263967  2.9220755   1.        ])
variable(8.558871)
(array(3., dtype=float32),)
[-0.28527933  5.844151   -6.1294303 ]
num dx1:variable(-0.2861023)
num dy1:variable(5.8174133)
num dt1:variable(-6.1035156)
[4.763546  3.3976624 0.       ]
num dx2:variable(4.863739)
num dy2:variable(3.4332275)
num dt2:variable(0.)
variable([3.7748961e-08 4.9999982e-01 2.3841858e-07])
variable(0.24999982)
variable([0.00199571 0.4002306  0.0400002 ])
variable(0.1601885)
variable([0.00484054 0.3210952  0.07126999])
variable(0.10312556)
variable([0.00758914 0.25813824 0.09577823])
variable(0.06669294)
variable([0.00983165 0.2079103  0.11502945])
variable(0.04332335)
variable([0.01144296 0.16773471 0.13017881])
variable(0.02826588)
variable([0.01244044 0.13552617 0.14211774])
variable(0.01852211)
variable([0.01290465 0.10965363 0.15153635])
variable(0.01219045)
variable([0.0129383  0.08883403 0.15897274])
variable(0.00805889)
variable([0.01264359 0.07205401 0.16484725])
variable(0.00535164)
varia

In [147]:
from chainer.utils import type_check
from chainer.utils import array


def _check_ndim(in_type, lower=1, upper=2):
    type_check.expect(
        in_type.ndim >= lower,
        in_type.ndim <= upper
    )
def _convert_type(in_type, vector_ndim=1):
    if in_type.ndim.eval() == vector_ndim:
        in_type = type_check.Variable(
            type_check.TypeInfo(in_type.shape.eval() + (1,),
                                in_type.dtype),
            '%s(1-D array)' % in_type.name)
    else:
        in_type.name = '%s(2-D array)' % in_type.name
    return in_type

def _matmul(a, b, transa=False, transb=False, transout=False):
    a = array.as_mat(a)
    b = array.as_mat(b)
    if transout:
        # (A B)^T = B^T A^T
        transa, transb = not transb, not transa
        a, b = b, a
    if transa:
        a = a.T
    if transb:
        b = b.T
    return a.dot(b)

class MatMul2(Function):

    def __init__(self, transa=False, transb=False):
        self.transa = transa
        self.transb = transb


    def forward(self, x):
        a, b = x
        return _matmul(a, b, transa=self.transa, transb=self.transb),

    def backward(self, x, gy):
        a, b = x
        print(gy)
        ga = _matmul(
            gy[0], b, transb=not self.transb, transout=self.transa
        ).reshape(a.shape)
        gb = _matmul(
            a, gy[0], transa=not self.transa, transout=self.transb
        ).reshape(b.shape)
        return ga, gb

def matmul2(a, b, transa=False, transb=False):
    """Computes the matrix multiplication of two arrays.

    Args:
        a (Variable): The left operand of the matrix multiplication.
            A 1-D array of shape ``(N,)`` is considered as an
            :math:`N \\times 1` matrix.
            A 2-D array of shape ``(M, N)`` is considered as an
            :math:`M \\times N` matrix.
        b (Variable): The right operand of the matrix multiplication.
            Its array is treated as a matrix in the same way as ``a``'s array.
        transa (bool): If ``True``, transpose ``a``.
        transb (bool): If ``True``, transpose ``b``.

    Returns:
        ~chainer.Variable: The result of the matrix multiplication as a 2-D
            array.
    """
    return MatMul2(transa=transa, transb=transb)(a, b)

In [113]:
x = Variable(np.array([[5, 2],[2, 1]], dtype = 'float'))
y = Variable(np.array([[3, 2],[-2, 1]], dtype = 'float'))
w = matmul2(x, y) 
print(w)

e = F.average(w)
e.backward()
print(x.grad)
print(y.grad)

variable([[11. 12.]
          [ 4.  5.]])
(array([[0.25, 0.25],
       [0.25, 0.25]]),)
[[5. 2.]
 [2. 1.]]
[[ 3.  2.]
 [-2.  1.]]
[[ 1.25 -0.25]
 [ 1.25 -0.25]]
[[1.75 1.75]
 [0.75 0.75]]


In [15]:
w


variable([[19.],
          [10.]])

In [29]:
e = F.matmul( F.transpose(w), w)
print(e)
e.backward()

variable([[461.]])


In [348]:
phases = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype='float32')
print(np.unwrap(phases))
phases = ( phases + np.pi) % (2 * np.pi ) - np.pi
print(phases)

[1. 2. 3. 4. 5. 6. 7. 8.]
[ 1.0000002   2.0000002   3.0000002  -2.2831852  -1.2831852  -0.28318524
  0.71681476  1.7168148 ]
